In [1]:
def txt_to_list(path):
    with open(path, 'r') as f:
        txt = str(f.read())
    txt_list = txt.split('\n')
    text_label_list = list()
    tmp = list()
    for line in txt_list:
        if line == '--------------------':
            text_label_list.append(tmp)
            tmp = list()
            continue
        if line == '':
            continue
        tmp.append(line)
    x = list()
    y = list()
    for text_label in text_label_list:
        text = text_label[0]
        text_list = [i for i in text]
        label = text_label[2:]
        label_list = ['O' for i in range(len(text_list))]
        for i in label:
            entity = i.split('\t')
            b = int(entity[1])
            label_list[b] = 'B-{}'.format(entity[-1])
            for j in range(int(entity[1])+1, int(entity[2])):
                label_list[j] = 'I-{}'.format(entity[-1])
        x.append(text_list)
        y.append(label_list)
    return [x, y]

In [2]:
x1, y1 = txt_to_list('../datasets/stage1/SampleData_deid.txt')

In [3]:
x2, y2 = txt_to_list('../datasets/stage2/train_1_update.txt')

In [4]:
x4, y4 = txt_to_list('../datasets/stage4/train_2.txt')

In [5]:
x = x1 + x2 + x4
len(x)

346

In [6]:
y = y1 + y2 + y4
len(y)

346

In [19]:
print(x[0])

['醫', '師', '：', '你', '有', '做', '超', '音', '波', '嘛', '，', '那', '我', '們', '來', '看', '報', '告', '，', '有', '些', '部', '分', '有', '紅', '字', '耶', '。', '民', '眾', '：', '紅', '字', '是', '甚', '麼', '意', '思', '？', '醫', '師', '：', '就', '是', '肝', '功', '能', '有', '比', '較', '高', '，', '肝', '功', '能', '6', '8', '，', '就', '是', '這', '個', 'A', 'L', 'P', '是', '6', '8', '，', '這', '樣', '比', '較', '高', '，', '正', '常', '應', '是', '5', '0', '以', '下', '，', '另', '外', '就', '是', '你', '之', '前', '說', '你', '有', 'B', '肝', '，', '但', '是', '你', 'B', '肝', '已', '經', '好', '了', '耶', '。', '民', '眾', '：', '它', '會', '自', '動', '修', '復', '阿', '。', '醫', '師', '：', '你', '有', '抗', '體', '了', '阿', '，', '所', '以', '你', 'B', '肝', '已', '經', '沒', '帶', '原', '了', '耶', '。', '民', '眾', '：', '我', '以', '前', '被', '關', '的', '時', '候', '，', '就', '有', '在', '固', '定', '驗', '血', '，', '那', '時', '候', '說', '有', 'B', '肝', '。', '醫', '師', '：', '阿', '你', '現', '在', '已', '經', '有', '保', '護', '的', '抗', '體', '了', '。', '但', '是', '你', '現', '在', '有', 'C', '肝', '。', '民', '眾', '：', 'C',

In [18]:
print(y[0])

['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-med_exam', 'I-med_exam', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-med_exam', 'I-med_exam', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'

In [7]:
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.2, random_state=42)
train_x, valid_x, train_y, valid_y = train_test_split(train_x, train_y, test_size=0.2, random_state=42)
print(len(train_x), len(train_y))
print(len(valid_x), len(valid_y))
print(len(test_x), len(test_y))

220 220
56 56
70 70


In [8]:
import kashgari
kashgari.config.use_cudnn_cell = True
from kashgari.tasks.labeling.models import BiLSTM_Model

model = BiLSTM_Model()
model.fit(train_x,
          train_y,
          valid_x,
          valid_y,
          epochs=10)
model.evaluate(test_x, test_y)
model.save('model/BiLSTM')

╭─────────────────────────────────────────────────────────────────────────╮
│ ◎ ○ ○ ░░░░░░░░░░░░░░░░░░░░░  Important Message  ░░░░░░░░░░░░░░░░░░░░░░░░│
├─────────────────────────────────────────────────────────────────────────┤
│                                                                         │
│              We renamed again for consistency and clarity.              │
│                   From now on, it is all `kashgari`.                    │
│  Changelog: https://github.com/BrikerMan/Kashgari/releases/tag/v1.0.0   │
│                                                                         │
│         | Backend          | pypi version   | desc           |          │
│         | ---------------- | -------------- | -------------- |          │
│         | TensorFlow 2.x   | kashgari 2.x.x | coming soon    |          │
│         | TensorFlow 1.14+ | kashgari 1.x.x |                |          │
│         | Keras            | kashgari 0.x.x | legacy version |          │
│           

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 4211)]            0         
_________________________________________________________________
layer_embedding (Embedding)  (None, 4211, 100)         172400    
_________________________________________________________________
layer_blstm (Bidirectional)  (None, 4211, 256)         235520    
_________________________________________________________________
layer_dropout (Dropout)      (None, 4211, 256)         0         
_________________________________________________________________
layer_time_distributed (Time (None, 4211, 28)          7196      
_________________________________________________________________
activation (Activation)      (None, 4211, 28)          0         
Total params: 415,116
Trainable params: 415,116
Non-trainable params: 0
_____________________________________________________

In [9]:
# loaded_model = kashgari.utils.load_model('model/BiLSTM')
# loaded_model.predict(test_x)[0]

In [10]:
import kashgari
from kashgari.tasks.labeling import BiLSTM_Model
from kashgari.embeddings import BERTEmbedding

bert_embed = BERTEmbedding('chinese_L-12_H-768_A-12',
                           task=kashgari.LABELING)
model = BiLSTM_Model(bert_embed)
model = BiLSTM_Model()
model.fit(train_x,
          train_y,
          valid_x,
          valid_y,
          epochs=10)
model.evaluate(test_x, test_y)
model.save('model/BERT_BiLSTM')

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 4211)]            0         
_________________________________________________________________
layer_embedding (Embedding)  (None, 4211, 100)         172400    
_________________________________________________________________
layer_blstm (Bidirectional)  (None, 4211, 256)         235520    
_________________________________________________________________
layer_dropout (Dropout)      (None, 4211, 256)         0         
_________________________________________________________________
layer_time_distributed (Time (None, 4211, 28)          7196      
_________________________________________________________________
activation_1 (Activation)    (None, 4211, 28)          0         
Total params: 415,116
Trainable params: 415,116
Non-trainable params: 0
_____________________________________________________

In [13]:
loaded_model = kashgari.utils.load_model('model/BERT_BiLSTM')
loaded_model.predict(test_x)